In [ ]:
# default_exp nlp.fasttext

# FastText - UMAP - HDBSCAN

> API details.

In [ ]:
import os
import time
import sys
import inspect

import pandas as pd
import numpy as np

import datetime

try:
    from fasttext import train_unsupervised
    import fasttext
except:
    from fastText import train_unsupervised
    import fastText
    
import umap.umap_ as umap
import hdbscan

import plotly.graph_objs as go
import plotly.offline as py
import plotly.graph_objs as go
from IPython.display import Image,display

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Get Data

In [ ]:
fasttext_data_folder = "../data/fasttext_data"
train_data = os.path.join(fasttext_data_folder,"text_file.text")

In [ ]:
text_df = pd.read_csv(train_data, names=["itemDesc"])
text_df.head()

,itemDesc
0,MINI DONUTS FOURRES ASSORTI X6
1,NAVETTE ASSORTIMENT X30
2,BRIOCHE MOUNA 400G
3,BRIOCHETTE ST GENIX X4 LOCAL
4,BRIOCHE ST GENIX LOCAL


## FastText training

In [ ]:
cbow_model = os.path.join(fasttext_data_folder,"model_cbow.bin")
skipgram_model = os.path.join(fasttext_data_folder,"model_skipgram.bin")


**train_unsupervised parameters**
    * input             # training file path (required)
    lr                # learning rate [0.1]
    dim               # size of word vectors [100]
    ws                # size of the context window [5]
    epoch             # number of epochs [5]
    minCount          # minimal number of word occurences [1]
    minCountLabel     # minimal number of label occurences [1]
    minn              # min length of char ngram [0]
    maxn              # max length of char ngram [0]
    neg               # number of negatives sampled [5]
    wordNgrams        # max length of word ngram [1]
    loss              # loss function {ns, hs, softmax, ova} [softmax]
    bucket            # number of buckets [2000000]
    thread            # number of threads [number of cpus]
    lrUpdateRate      # change the rate of updates for the learning rate [100]
    t                 # sampling threshold [0.0001]
    label             # label prefix ['__label__']
    verbose           # verbose [2]
    pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []

In [ ]:
model_to_use = "skipgram"

if model_to_use == "skipgram":
    # Skipgram model :
    model_skipgram = train_unsupervised(train_data, model='skipgram')
    model_skipgram.save_model(skipgram_model)

else:
    #Cbow model :
    model_cbow = train_unsupervised(train_data, model='cbow')
    model_cbow.save_model(cbow_model)

In [ ]:
# Loading of the fastext pretained model
model_skipgram=fasttext.load_model(skipgram_model)

In [ ]:
model_skipgram.get_dimension()

100

In [ ]:
# The get_sentence_vector takes all vectors for all the words in the query, divide each of them by their respective norm, and then average all vectors together
def query_to_vector(col_query, model_fastText):
    vector = col_query.apply(lambda x:model_fastText.get_sentence_vector(x.replace('\n',' ')))
    return vector

In [ ]:
%timeit text_df['vector'] = query_to_vector(text_df['itemDesc'], model_skipgram)
del model_skipgram # We do not need the pretrained-vector in memory

13.1 ms ± 831 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
text_df.head(10)

,itemDesc,vector
0,MINI DONUTS FOURRES ASSORTI X6,"[0.02189265, 0.10787461, -0.014234907, -0.0226..."
1,NAVETTE ASSORTIMENT X30,"[0.014975755, 0.04644253, -0.04249201, -0.0160..."
2,BRIOCHE MOUNA 400G,"[-0.0522879, 0.001154994, 0.0076559577, -0.075..."
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.009413259, -0.017262919, -0.007244551, -0...."
4,BRIOCHE ST GENIX LOCAL,"[-0.01687116, -0.03279358, 0.000291368, -0.017..."
5,NAVETTE X20,"[0.026375797, -0.00427916, -0.047491483, -0.00..."
6,TARTE AU SUCRE 6P LOCAL,"[-0.022558345, -0.087581225, -0.011884119, 0.0..."
7,MINI SUISSE X8,"[0.09554361, 0.010526974, 0.14405009, -0.13391..."
8,4 P RAIS+4 P CHOC.+4 CROIS. PB,"[0.012128612, 0.0345035, -0.043019157, 0.03134..."
9,X5 POCHE TALOAK,"[0.011469854, 0.007848586, 0.007189207, -0.033..."


## Dimension reduction with UMAP

In [ ]:
# We regroup all the vectors as a numpy array
vecs=text_df.vector.values
vecs=np.stack(vecs, axis=0)
vecs.shape

(583, 100)

In [ ]:
col_names = ["FT_"+str(x) for x in range(0,100)]
print(len(col_names))
test = pd.DataFrame(vecs, columns=col_names)

100


In [ ]:
pd.concat([text_df,test], axis=1)

,itemDesc,vector,FT_0,FT_1,FT_2,FT_3,FT_4,FT_5,FT_6,FT_7,FT_8,FT_9,FT_10,FT_11,FT_12,FT_13,FT_14,FT_15,FT_16,FT_17,FT_18,FT_19,FT_20,FT_21,FT_22,FT_23,FT_24,FT_25,FT_26,FT_27,FT_28,FT_29,FT_30,FT_31,FT_32,FT_33,FT_34,FT_35,FT_36,FT_37,FT_38,FT_39,FT_40,FT_41,FT_42,FT_43,FT_44,FT_45,FT_46,FT_47,FT_48,FT_49,FT_50,FT_51,FT_52,FT_53,FT_54,FT_55,FT_56,FT_57,FT_58,FT_59,FT_60,FT_61,FT_62,FT_63,FT_64,FT_65,FT_66,FT_67,FT_68,FT_69,FT_70,FT_71,FT_72,FT_73,FT_74,FT_75,FT_76,FT_77,FT_78,FT_79,FT_80,FT_81,FT_82,FT_83,FT_84,FT_85,FT_86,FT_87,FT_88,FT_89,FT_90,FT_91,FT_92,FT_93,FT_94,FT_95,FT_96,FT_97,FT_98,FT_99
0,MINI DONUTS FOURRES ASSORTI X6,"[0.02189265, 0.10787461, -0.014234907, -0.0226...",0.021893,0.107875,-0.014235,-0.022613,0.028204,0.087868,-0.041379,0.034871,-0.038356,0.059184,-0.016758,-0.067529,-0.010123,0.041630,0.018600,-0.033242,-0.008634,-0.075442,-0.076242,-0.031079,-0.031463,-0.010116,-0.019682,0.002145,-0.017118,0.034094,0.046754,0.013679,-0.010289,-0.016059,-0.001146,-0.051525,0.071037,-0.055208,0.027236,-0.115201,-0.011364,-0.041341,-0.006985,-0.002378,0.018476,-0.029233,-0.059060,-0.060528,0.035874,-0.015067,0.071754,0.067342,0.024840,0.022026,-0.020118,-0.039323,-0.076980,3.325633e-02,0.004085,0.007250,0.005760,-0.037471,-0.006596,0.033018,0.109290,-0.067883,-0.014137,-0.016261,0.012785,-0.035506,-0.030905,-0.034955,-0.025966,-0.051804,0.035805,0.080714,-0.037328,0.019347,-0.025617,0.018011,0.074893,0.010634,-0.003897,0.003587,0.047191,-0.005397,0.038612,-0.001835,0.001973,-0.011140,-0.029869,-0.032659,0.048333,0.006200,-0.016012,-0.074827,0.016927,-0.076514,0.034602,-0.009376,0.104716,-0.032211,-0.043635,-0.098906
1,NAVETTE ASSORTIMENT X30,"[0.014975755, 0.04644253, -0.04249201, -0.0160...",0.014976,0.046443,-0.042492,-0.016020,0.060589,-0.001802,-0.007963,0.154584,-0.047432,0.001200,0.032157,-0.059554,0.000718,0.001321,0.161345,-0.091178,-0.071525,0.090844,-0.095655,0.029134,-0.081196,0.044388,-0.016175,0.024270,-0.010981,-0.078220,0.084995,0.005192,-0.034364,-0.063775,-0.033751,0.063376,0.003899,-0.017413,-0.007193,-0.062290,-0.106323,-0.081042,-0.023460,-0.035445,0.023878,0.077496,0.040673,-0.087632,-0.048149,0.029242,-0.020232,-0.013576,-0.000162,-0.003269,0.059127,0.015152,-0.114693,-5.696078e-02,-0.020701,-0.028144,-0.116559,0.071883,-0.002630,-0.021492,-0.057264,-0.058594,0.019816,-0.075325,-0.025922,0.038827,-0.069613,-0.000352,0.081464,-0.058647,0.071187,-0.005142,-0.045691,-0.052592,-0.041623,0.094520,0.106152,-0.012685,-0.102757,0.048036,-0.119291,0.007339,0.008281,0.128907,0.028417,-0.079641,0.045603,-0.008531,-0.040964,-0.006577,-0.094621,-0.047710,0.011104,-0.078821,0.081602,0.060157,0.022344,-0.001476,-0.122729,-0.096902
2,BRIOCHE MOUNA 400G,"[-0.0522879, 0.001154994, 0.0076559577, -0.075...",-0.052288,0.001155,0.007656,-0.075826,0.011916,0.064263,-0.018506,0.027885,0.002566,-0.022193,-0.051448,0.038506,-0.013118,-0.007563,0.048938,0.066030,0.004098,0.000231,0.013453,-0.028538,0.067185,0.081719,-0.071251,0.062017,-0.029551,-0.018661,-0.022595,-0.008173,0.016262,-0.010919,-0.005681,0.020875,-0.018665,-0.064767,-0.049660,0.037674,-0.018416,-0.010805,-0.148198,-0.020311,0.046140,0.063624,-0.055314,0.033542,-0.024620,0.047605,0.064045,0.028681,0.077954,-0.031763,-0.018746,-0.010235,-0.048626,-2.323270e-02,-0.143056,-0.029130,0.026990,0.028632,0.018002,0.063775,-0.027766,-0.055045,-0.087512,-0.024886,-0.017916,-0.036391,0.023741,0.018399,-0.055502,0.056110,0.060544,0.017170,-0.041984,-0.019636,-0.081729,-0.075883,0.020914,0.087724,-0.014154,0.004644,-0.041628,0.026734,0.096230,0.105805,-0.008227,-0.062245,-0.015890,0.058891,-0.071408,-0.023343,0.039281,-0.050999,-0.062892,-0.080430,-0.132587,-0.143349,-0.024183,-0.058106,0.012511,-0.110218
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.009413259, -0.017262919, -0.007244551, -0....",-0.009413,-0.017263,-0.007245,-0.020935,0.027331,0.004281,-0.031993,0.004024,-0.055252,-0.096062,-0.016239,0.009742,-0.006918,-0.030301,0.017237,-0.032608,0.027304,-0.014444,-0.017272,0.0

In [ ]:
fit = umap.UMAP(n_neighbors=30,min_dist=0.1,n_components=3,metric='cosine',random_state=42)
%time u = fit.fit_transform(vecs)

text_df['x']=u[:,0]
text_df['y']=u[:,1]
text_df['z']=u[:,2]

CPU times: user 5.64 s, sys: 279 ms, total: 5.92 s
Wall time: 7.26 s


In [ ]:
u.shape

(583, 3)

## 3D Vizualisation

In [ ]:
# This is the function to plot the queries in the embedding space.
# Here we reduce the embedding to a 3 dimensionnal space
def plot_cluster(df,iscolored=False,name='',interactive=True):
    
    if interactive:
        if iscolored:
            color=df['cluster'].values
        else: color = df['x']
        trace1 = go.Scatter3d(
            x=df['x'],
            y=df['y'],
            z=df['z'],

            mode='markers',
            marker=dict(
                size=3,
                color=color,                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.3
            ),
            text=color
        )

        data = [trace1]
        layout = go.Layout(
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            )
        )
        fig = go.Figure(data=data, layout=layout)
        file='../data/fasttext_data/'+name+'.html'
        py.iplot(fig, filename=file)
        py.plot(fig, filename=file,auto_open=False)

In [ ]:
# dont forget to create the folder "images" in your directory
is_interactive=True
filename='umap_embedding_description'

if is_interactive:
    plot_cluster(text_df,False,filename)
else:
    display(Image('../data/fasttext_data/'+filename+'.png'))

## Clustering with HDBSCAN

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5,min_samples=5)

In [ ]:
clusterer

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='euclidean', min_cluster_size=5, min_samples=5, p=None,
        prediction_data=False)

In [ ]:
%time text_df['cluster']= clusterer.fit_predict(u)

CPU times: user 21.3 ms, sys: 2.69 ms, total: 24 ms
Wall time: 21.7 ms


In [ ]:
text_df.groupby(['cluster'])['itemDesc'].apply(list).apply(len).sort_values(ascending=False)[0:20]

cluster
-1     189
 21     76
 11     27
 22     27
 0      21
 26     21
 1      20
 6      20
 7      16
 13     14
 10     14
 12     13
 14     11
 2      10
 5       9
 16      9
 3       9
 4       9
 8       9
 18      8
Name: itemDesc, dtype: int64

In [ ]:
# The cluster number -1 means actually that the algorithm considered it as noise.
# So we will remove the noise
denoised = text_df[text_df.cluster!=-1]

In [ ]:
denoised

,itemDesc,vector,x,y,z,cluster
0,MINI DONUTS FOURRES ASSORTI X6,"[0.02189265, 0.10787461, -0.014234907, -0.0226...",-0.399989,8.043009,0.023928,15
2,BRIOCHE MOUNA 400G,"[-0.0522879, 0.001154994, 0.0076559577, -0.075...",3.646831,8.561210,-2.679703,21
3,BRIOCHETTE ST GENIX X4 LOCAL,"[-0.009413259, -0.017262919, -0.007244551, -0....",3.223323,9.836374,-1.557101,22
4,BRIOCHE ST GENIX LOCAL,"[-0.01687116, -0.03279358, 0.000291368, -0.017...",3.437589,9.720731,-1.623649,22
7,MINI SUISSE X8,"[0.09554361, 0.010526974, 0.14405009, -0.13391...",0.549603,8.045481,0.598263,8
8,4 P RAIS+4 P CHOC.+4 CROIS. PB,"[0.012128612, 0.0345035, -0.043019157, 0.03134...",1.030909,7.811539,-2.037988,7
10,X4 DONUTS ASSORTIS -40%MG,"[-0.013870355, 0.11059206, -0.050391905, 0.003...",-1.079710,8.982529,0.299735,0
12,BRIOCHE NANTERRE PB 200G,"[-0.05712038, -0.08311469, 0.025220992, -0.002...",2.983588,7.975422,-2.491688,21
14,MAXI BEIGNET POMME X2,"[-0.042241365, -0.040620063, -0.015010707, -0....",0.272990,9.709663,-0.315733,11
17,MINI VIENNOISERIE X10,"[0.047347154, -0.05774416, 0.05095637, -0.0957...",0.392011,7.782959,-0.272686,14


In [ ]:
# Number of clusters
print('Total number of clusters: '+str(len(denoised.cluster.unique())))

Total number of clusters: 27


In [ ]:
clusters=denoised.groupby(['cluster'])['itemDesc'].apply(list)

In [ ]:
cluster_i = clusters[0]
print(len(cluster_i))
cluster_i

21


['X4 DONUTS ASSORTIS -40%MG',
 'DONUTS PARTY X4',
 'DONUTS FOURRES CHOCOLAT X4',
 'DONUTS CHOCOLAT X4',
 'DONUTS RAYES ASSORTIS X4',
 'DONUTS FOURRES FRAISE X4',
 'DONUTS -40%MG DECORS CHOCO X4',
 'DONUTS -40%MG NATURE X4',
 '76G MINI DONUTS MILKA  X4',
 'DONUTS SUCRE X4 LOCAL',
 'DONUTS CHOCOLAT X4 LOCAL',
 'MIX BOX DONUT X4',
 'DONUTS HALLOWEEN X4',
 'X4 DONUTS MILKA',
 'DONUTS BLACK & GOLD X4',
 'DONUTS FRAISE SIMPSON X4',
 'DONUTS CHOCOLAT SIMPSON X4',
 'DONUTS ASSORTIS X8',
 'DONUTS MILKA X4',
 'DONUTS ASSORTIS X4',
 'DONUTS MILKA X4']

In [ ]:
for i, cluster_i in enumerate(clusters):
    print("---------")
    print("cluster {} size:{}".format(i,len(cluster_i)))
    print(cluster_i)

---------
cluster 0 size:21
['X4 DONUTS ASSORTIS -40%MG', 'DONUTS PARTY X4', 'DONUTS FOURRES CHOCOLAT X4', 'DONUTS CHOCOLAT X4', 'DONUTS RAYES ASSORTIS X4', 'DONUTS FOURRES FRAISE X4', 'DONUTS -40%MG DECORS CHOCO X4', 'DONUTS -40%MG NATURE X4', '76G MINI DONUTS MILKA  X4', 'DONUTS SUCRE X4 LOCAL', 'DONUTS CHOCOLAT X4 LOCAL', 'MIX BOX DONUT X4', 'DONUTS HALLOWEEN X4', 'X4 DONUTS MILKA', 'DONUTS BLACK & GOLD X4', 'DONUTS FRAISE SIMPSON X4', 'DONUTS CHOCOLAT SIMPSON X4', 'DONUTS ASSORTIS X8', 'DONUTS MILKA X4', 'DONUTS ASSORTIS X4', 'DONUTS MILKA X4']
---------
cluster 1 size:20
['KIDS CRUNCHY DONUT X2', 'DONUTS DAIM BLISTER X2', 'DONUTS DAIM X2', 'DONUTS DAIM FOURRES X2', 'DONUTS OREO BLISTER X2', 'DONUT FOURRE CACAO X2', 'DONUT FOURRE FRAISE X2', 'DONUT OREO FOURRE X2', 'DONUTS CHOCOLAT X2', 'DONUTS SUCRE X2', 'DONUT WHITE X2', 'DONUTS GOLDEN WINTER X2', 'DONUTS SPECULOOS X2', 'DONUTS CONFETTIS X2', 'DONUTS CHOCOLAT CAKE X2', '75G DONUT ZEBRE FOURRE CACAO', 'DONUTS OREO X2', 'DONUT FOUR

## TF-IDF into the clusters

In [ ]:
def get_top_grams(cluster_j):
    tokens = nltk.word_tokenize(" ".join(cluster_j))

    #Create your bigrams
    tgs = nltk.trigrams(tokens)
    bgs = nltk.bigrams(tokens)

    #compute frequency distribution for all the bigrams in the text
    fdist = nltk.FreqDist(tgs)
    trigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
    fdist = nltk.FreqDist(bgs)
    bigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
    fdist = nltk.FreqDist(tokens)
    onegram = fdist.most_common(1)[0][0],fdist.most_common(1)[0][1]

    return trigram, bigram, onegram

In [ ]:
# Cluster description
for i, cluster_i in enumerate(clusters):
    print("---------")
    print("cluster {} size:{}".format(i,len(cluster_i)))
    print(get_top_grams(cluster_i))

---------
cluster 0 size:21


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(('X 4 LOCAL', 3), ('LOCAL COQUILLE', 6), ('LOCAL', 20))


In [ ]:
tokens = nltk.word_tokenize(" ".join(cluster_i))

#Create your bigrams
tgs = nltk.trigrams(tokens)
bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(tgs)
trigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
fdist = nltk.FreqDist(bgs)
bigram = " ".join(fdist.most_common(1)[0][0]), fdist.most_common(1)[0][1]
fdist = nltk.FreqDist(tokens)
onegram = fdist.most_common(1)[0][0],fdist.most_common(1)[0][1]

print(trigram, bigram, onegram)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

('X 4 LOCAL', 3) ('LOCAL COQUILLE', 6) ('LOCAL', 20)


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()